# Importar base de datos

In [7]:
import pandas as pd
import numpy as np
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Cargar la base de datos
data = pd.read_csv('Datos_chatbot.csv', encoding='UTF-8')

# Extraer preguntas y respuestas
X = data['Preguntas']
Y = data['Respuestas']


# Preprocesamiento del texto

In [8]:
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Descargar stopwords de NLTK
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('spanish'))
stemmer = SnowballStemmer('spanish')

def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar puntuación
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Eliminar dígitos
    text = re.sub(r'\d+', '', text)
    # Tokenizar y eliminar stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    # Stemming
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

X_processed = X.apply(preprocess_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Vectorizacion del texto

In [9]:
# Vectorización usando TF-IDF
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X_processed)

# Definir una función para encontrar la respuesta más similar
def get_response(user_query):
    user_query_processed = preprocess_text(user_query)
    user_query_tfidf = vectorizer.transform([user_query_processed])

    # Calcular similaridad coseno
    similarities = cosine_similarity(user_query_tfidf, X_tfidf)
    closest_idx = np.argmax(similarities, axis=1)[0]

    return Y.iloc[closest_idx]

# Prueba del chatbot
print(get_response("¿Qué documentos son necesarios para la inscripción?"))


Los documentos requeridos son acta de grado y/o diploma, Pruebas Saber 11, documento de identidad.


# Interaccion con el Chatbot

In [10]:
def chat():
    print("Chatbot iniciado. Escribe 'salir' para terminar la conversación.")
    while True:
        user_input = input("Tú: ")
        if user_input.lower() == 'salir':
            print("Chatbot: ¡Adiós!")
            break
        response = get_response(user_input)
        print(f"Chatbot: {response}")

# Iniciar el chatbot
chat()


Chatbot iniciado. Escribe 'salir' para terminar la conversación.
Tú: fechas
Chatbot: Las fechas son entre septiembre, octubre y noviembre / abril, mayo y junio.
Tú: salir
Chatbot: ¡Adiós!
